1. **age**
2. **sex**
3. **cp** chest pain type (4 values)
4. **trestbps** resting blood pressure
5. **chol** serum cholestoral in mg/dl
6. **fbs** fasting blood sugar > 120 mg/dl
7. **restecg** resting electrocardiographic results (values 0,1,2)
8. **thalach** maximum heart rate achieved
9. **exang** exercise induced angina
10. **oldpeak**  ST depression induced by exercise relative to rest
11. **slope** of the peak exercise ST segment
12. **ca** number of major vessels (0-3) colored by flourosopy
13. **thal** 3 = normal; 6 = fixed defect; 7 = reversable defect
14. **target**  1 or 0

In [1]:
# eta media per sesso
# frequenca di chest pain per sesso
# colesterolo medio per sesso diviso per fasce di età (<50,50-60,>70)
# il colesterolo e la pressione tra persone target e non target è differente?
# frequenza del tipo di slope tra persone target e non target
# frequenza di angina da esercizio tra persone target e non target divisi per fasce di età e sesso
# correlazione tra colesterolo nel sangue e pressione tra persone target e non target

In [2]:
import re

def reader(percorso):
    f = open(percorso)
    dataset = dict()
    for i,line in enumerate(f):
        if i==0:
            header = line.split(",")
            for h in header:
                h = re.sub('[\x80-\xFF]','',h)
                if "target" in h:
                    h = h[:-1]
                dataset[h] = list()
        else:
            line = line.split(",")
            for h,k in enumerate(dataset.keys()):

                if k == "oldpeak":
                    line[h] = float(line[h])
                elif k == "target":
                    line[h] = line[h][:-1]
                elif k in ["age", "trestbps", "chol", "thalach"]:
                    line[h] = int(line[h])

                dataset[k].append(line[h])

    f.close()
    
    return dataset





In [3]:
pathtofile = "C:\\Users\\oldboy\\Documents\\GitHub\\corso_python\\dataset_2\\heart.csv"
dataset = reader(pathtofile)
dataset.keys()


dict_keys(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'])

In [4]:
#eta media in base al sesso
import numpy as np

eta_maschi = list()
eta_femmine = list()

for i,s in enumerate(dataset["sex"]):
    if s =="1":
        eta_femmine.append(dataset["age"][i])
    else:
        eta_maschi.append(dataset["age"][i])
        
print("femmine",np.mean(eta_femmine))
print("maschi",np.mean(eta_maschi))


femmine 53.75845410628019
maschi 55.677083333333336


In [66]:
eta = dict()
eta["maschi"]=list()
eta["femmine"]=list()

for i,s in enumerate(dataset["sex"]):
    if s=="1":
        eta["femmine"].append(dataset["age"][i])
    else:
        eta["maschi"].append(dataset["age"][i])
    
    
print("maschi: ",np.mean(eta["maschi"]))
print("femmine: ",np.mean(eta["femmine"]))

maschi:  55.677083333333336
femmine:  53.75845410628019


In [73]:
cat = "slope"
cont = "oldpeak"
eta = dict()
val = set(dataset[cat])
for k in val:
    eta[k]=list()
    
for i,s in enumerate(dataset[cat]):
    eta[s].append(dataset[cont][i])
    
for k in eta.keys():
    eta[k] = np.mean(eta[k])
    
eta

{'1': 1.4285714285714286, '0': 2.7142857142857144, '2': 0.4084507042253521}

In [86]:

class Person():
    
    def __init__(self,name,age):
        self.name = name
        self.autosex()
        self.age = age
    
    def cresce(self):
        self.age+=1
        
    def autosex(self):
        if self.name[-1]=="a":
            self.sex = "f"
        elif self.name[-1]=="o":
            self.sex = "m" 
        
    def stampa(self):
        print("Nome: ",self.name, "Sex: ",self.sex, "Age: ",self.age)



In [87]:
soggetto1 = Person("Sandro",18)
soggetto2 = Person("Franca",22)

soggetto1.stampa()
soggetto1.cresce()
soggetto1.stampa()

Nome:  Sandro Sex:  m Age:  18
Nome:  Sandro Sex:  m Age:  19


In [85]:
soggetto1.name[-1]

'o'

In [11]:
from collections import Counter

dict(Counter(["2","4","2","4","3","1"]))

{'2': 2, '4': 2, '3': 1, '1': 1}

In [15]:
# frequenza di chest pain per sesso

sex_keys = list(set(dataset["sex"]))
cp_keys = list(set(dataset["cp"]))
results = dict()
for k in sex_keys:
    results[k] = list()
for i,s in enumerate(dataset["sex"]):
    results[s].append(dataset["cp"][i])
    
for s in results:
    #temp = dict()
    #for k in cp_keys:
    #    temp[k] = 0
    #for c in results[s]:
    #    temp[c]+=1
    #results[s]=temp
    results[s]= dict(Counter(results[s]))
        
results

{'1': {'3': 19, '2': 52, '1': 32, '0': 104},
 '0': {'1': 18, '0': 39, '2': 35, '3': 4}}

In [17]:
results["1"]["0"]

104

In [21]:
a = ["a","b"]
b = a.copy()
b[1]="a"
a

['a', 'b']

In [23]:
a = 1
b = a
b = 2
a

1

In [48]:
string = 'ciao,"franco,gianni,pamela,aurelia",unni,cracovia,"10,1000",primavera'

virgo = False
for i,s in enumerate(string):
    if s == '"':
        virgo= not virgo  
    if virgo and s==",":
        string = string[:i]+";"+string[1+i:]
    
string  

'ciao,"franco;gianni;pamela;aurelia",unni,cracovia,"10;1000",primavera'

In [32]:
# frequenza di angina da esercizio tra persone target e non target divisi per fasce di età e sesso
import numpy as np
import itertools
from collections import Counter

dataset["age_group"]=list()

for i in dataset["age"]:
    if i<50:
        dataset["age_group"].append("young")
    elif i>=50 and i<=60:
        dataset["age_group"].append("interm")
    else:
        dataset["age_group"].append("old")
        

sessi = set(dataset["sex"])
tar = set(dataset["target"])
agroup = set(dataset["age_group"])

exang = np.array(dataset["exang"])
res = dict()

for s in sessi:
    res[s]=dict()
    for t in tar:
        res[s][t]=dict()
        for a in agroup:
            res[s][t][a]=dict()

for s,t,a in list(itertools.product(sessi,tar,agroup)):
    res[s][t][a] = dict(Counter(exang[(np.array(dataset["age_group"])==a) & (np.array(dataset["sex"])==s) & (np.array(dataset["target"])==t)]))
res

{'1': {'1': {'old': {'0': 10, '1': 3},
   'interm': {'0': 32, '1': 9},
   'young': {'0': 36, '1': 3}},
  '0': {'old': {'1': 17, '0': 15},
   'interm': {'1': 33, '0': 24},
   'young': {'0': 13, '1': 12}}},
 '0': {'1': {'old': {'0': 20, '1': 2},
   'interm': {'1': 2, '0': 25},
   'young': {'0': 19, '1': 4}},
  '0': {'old': {'0': 7, '1': 5},
   'interm': {'1': 8, '0': 3},
   'young': {'1': 1}}}}

In [33]:
list(itertools.product(sessi,tar,agroup))

[('1', '1', 'old'),
 ('1', '1', 'interm'),
 ('1', '1', 'young'),
 ('1', '0', 'old'),
 ('1', '0', 'interm'),
 ('1', '0', 'young'),
 ('0', '1', 'old'),
 ('0', '1', 'interm'),
 ('0', '1', 'young'),
 ('0', '0', 'old'),
 ('0', '0', 'interm'),
 ('0', '0', 'young')]